## 라이브러리 및 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:\\Python\\NonSmoking_Project\\data\\금연구역지정현황_서구_주소분리.csv', encoding='cp949')
df

,광역시,구,구분,시설명,주소
0,대전광역시,서구,일반음식점 및 휴게음식점,레몽이퐁당,가수원동 1001
1,대전광역시,서구,일반음식점 및 휴게음식점,소쿠리족발보쌈,가수원동 1002
2,대전광역시,서구,일반음식점 및 휴게음식점,남다른맥주가게,가수원동 1003
3,대전광역시,서구,일반음식점 및 휴게음식점,비어뱅크,가수원동 1004
4,대전광역시,서구,일반음식점 및 휴게음식점,구이구이삼겹살,가수원동 1005
...,...,...,...,...,...
15058,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 588-1
15059,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-0
15060,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-10
15061,대전광역시,서구,연면적 1천제곱미터 이상 건축물,1동,외국 610-11


In [3]:
import requests
import json

In [84]:
rest_api_key = "616734aa8214dbf958cfaebd6bd41aec"

In [85]:
class KakaoLocalAPI:
    """
    Kakao Local API Controller
    """
    def __init__(self, rest_api_key):
        """
        Rest API key 초기화 및 기능 별 url 설정
        """
        
        # REST API key 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}
        
        # 01 주소검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        
    def search_address(self, query, analyze_type = None, page = None, size = None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}
    
        if analyze_type != None:
            params["analyze_type"]=f"{analyze_type}"
    
        elif page != None:
            params['page'] = f"{page}"
    
        elif size != None:
            params['size'] = f"{size}"
    
        res = requests.get(self.URL_01, headers = self.headers, params = params)
        document = json.loads(res.text)
    
        return document

In [86]:
kakao = KakaoLocalAPI(rest_api_key)

## 데이터 관찰

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15063 entries, 0 to 15062
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   광역시     15063 non-null  object
 1   구       15063 non-null  object
 2   구분      15063 non-null  object
 3   시설명     13945 non-null  object
 4   주소      15063 non-null  object
dtypes: object(5)
memory usage: 588.5+ KB


In [8]:
df

,광역시,구,구분,시설명,주소
0,대전광역시,서구,일반음식점 및 휴게음식점,레몽이퐁당,가수원동 1001
1,대전광역시,서구,일반음식점 및 휴게음식점,소쿠리족발보쌈,가수원동 1002
2,대전광역시,서구,일반음식점 및 휴게음식점,남다른맥주가게,가수원동 1003
3,대전광역시,서구,일반음식점 및 휴게음식점,비어뱅크,가수원동 1004
4,대전광역시,서구,일반음식점 및 휴게음식점,구이구이삼겹살,가수원동 1005
...,...,...,...,...,...
15058,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 588-1
15059,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-0
15060,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-10
15061,대전광역시,서구,연면적 1천제곱미터 이상 건축물,1동,외국 610-11


## 데이터 전처리
[순서]
- 카카오맵 API에서 검색되는 주소와 그렇지 않은 주소를 각각 df, df_error로 1차 분리
- 이후 df_error 데이터 정제 후 유효하지 않는 데이터로 판단되는 행들을 삭제
- 삭제하고 남은 df_error를 기존 df와 합한 뒤, 위경도 추출

### 유효하지 않은 주소 데이터 분리

In [9]:
add_list = df['주소']

In [10]:
error_index = []

for i in range(len(add_list)):
    result_02 = kakao.search_address(add_list[i])
    if len(result_02['documents']) == 0:
        error_index.append(i)

In [11]:
len(error_index)
# 검색되지 않는 주소 데이터 955개 존재

955

In [12]:
df_error = df.iloc[error_index]
df_error # 1차 정제: 유효하지 않은 주소 데이터

,광역시,구,구분,시설명,주소
6,대전광역시,서구,일반음식점 및 휴게음식점,뚱보집가수원점,가수원동 1008 3
12,대전광역시,서구,일반음식점 및 휴게음식점,바담식탁,가수원동 1019
13,대전광역시,서구,일반음식점 및 휴게음식점,왕돌구이삼겹살왕삼이,가수원동 1020 1층
14,대전광역시,서구,일반음식점 및 휴게음식점,푸른바다참치연어,가수원동 1023
15,대전광역시,서구,일반음식점 및 휴게음식점,못난이꽈배기가수원점,가수원동 1024 RiverTree
...,...,...,...,...,...
15058,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 588-1
15059,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-0
15060,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-10
15061,대전광역시,서구,연면적 1천제곱미터 이상 건축물,1동,외국 610-11


In [13]:
df = df.drop(index = error_index, axis=0)
df # 1차 정제: 유효한 주소 데이터

,광역시,구,구분,시설명,주소
0,대전광역시,서구,일반음식점 및 휴게음식점,레몽이퐁당,가수원동 1001
1,대전광역시,서구,일반음식점 및 휴게음식점,소쿠리족발보쌈,가수원동 1002
2,대전광역시,서구,일반음식점 및 휴게음식점,남다른맥주가게,가수원동 1003
3,대전광역시,서구,일반음식점 및 휴게음식점,비어뱅크,가수원동 1004
4,대전광역시,서구,일반음식점 및 휴게음식점,구이구이삼겹살,가수원동 1005
...,...,...,...,...,...
15051,대전광역시,서구,신고체육시설,도마2체육관,흑석동 950
15052,대전광역시,서구,일반음식점 및 휴게음식점,흑석만두찐빵,흑석동 968
15053,대전광역시,서구,일반음식점 및 휴게음식점,흑석한우마을,흑석동 975 1층
15054,대전광역시,서구,담배판매업소 및 담배자동판매기,기성농협,흑석동 980호


### 주소에 '외국' 포함된 경우

In [14]:
df_error.loc[df_error['주소'].str.contains('외국')]

,광역시,구,구분,시설명,주소
15057,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 484-0
15058,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 588-1
15059,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-0
15060,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-10
15061,대전광역시,서구,연면적 1천제곱미터 이상 건축물,1동,외국 610-11
15062,대전광역시,서구,연면적 1천제곱미터 이상 건축물,NaN,외국 610-7


In [15]:
df_error = df_error.drop([15057, 15058, 15059, 15060, 15061, 15062], axis=0)

### 호 정보 포함된 경우의 데이터 정제

In [16]:
error_index_ho = df_error[df_error['주소'].str.contains('호')].index.tolist()

In [17]:
len(error_index_ho)

# 검색되지 않는 주소의 대부분이 호 정보를 포함하고 있음
# 자료의 개수가 많아 개별적인 접근이 어려움

552

In [18]:
for i in error_index_ho:
    if '호' in df_error['주소'][i].split(' '): 
        print([df_error['주소'][i], i])

['가수원동 호 375', 604]
['갈마동 호 309-3번지', 2020]
['갈마동 호 404-2', 2021]
['괴정동 호 11-8', 4686]
['괴정동 호 85-10', 4687]
['내동 호 13-19', 5034]
['도마동 호 100-28 ', 6154]
['도마동 호 50-6', 6155]
['둔산동 1119번지 104. 호', 7629]
['둔산동 호 10브럭 가람아파트상가 지하1호 ', 9331]
['둔산동 호 1262 ', 9332]
['둔산동 호 19-1브럭 벽산삼익아파트 상가106호 ', 9333]
['둔산동 호 국화우성아파트상가 103호 택시개발사업지구 ', 9334]
['복수동 호 삼익목화C상가5호 ', 11285]
['용문동 호 225-7', 11820]
['탄방동 호 1479-2 ', 14939]
['탄방동 호 1481-2 ', 14940]
['흑석동 호 15블럭 1롯트 ', 15056]


In [19]:
for i in error_index_ho:
    if '호' in df_error['주소'][i].split(' '):
        df_error['주소'][i] = df_error['주소'][i].replace('호 ', '') #'호'만 들어간 주소:'호' 제거
        print(df_error['주소'][i])

가수원동 375
갈마동 309-3번지
갈마동 404-2
괴정동 11-8
괴정동 85-10
내동 13-19
도마동 100-28 
도마동 50-6
둔산동 1119번지 104. 호
둔산동 10브럭 가람아파트상가 지하1
둔산동 1262 
둔산동 19-1브럭 벽산삼익아파트 상가106
둔산동 국화우성아파트상가 103택시개발사업지구 
복수동 삼익목화C상가5
용문동 225-7
탄방동 1479-2 
탄방동 1481-2 
흑석동 15블럭 1롯트 


In [20]:
for i in error_index_ho:
    for j in df_error['주소'][i].split(' '):
        if '번지' in j:
            print(df_error['주소'][i])

가수원동 656번지 27호
가수원동 656번지 62호
가장동 27번지 18호
가장동 30번지 12호
가장동 42번지 15호 1층
갈마1동 1427번지 102호
갈마1동 262번지 12호
갈마1동 262번지 4호
갈마1동 264번지 5호
갈마1동 285번지 8호
갈마1동 290번지 12호 1층
갈마1동 307번지 27호 102호
갈마1동 316번지 4호
갈마1동 366번지 23호 1층
갈마2동 343번지 27호
갈마2동 351번지 2호 한국도로공사조합아파트상가 104호
갈마2동 394번지 18호 1층
갈마2동 398번지 4호
갈마2동 404번지 9호
갈마동 261번지 14호 .261-17
갈마동 261번지 26호
갈마동 263번지 29호
갈마동 265번지 34호
갈마동 266번지 85호 102호
갈마동 271번지 1호
갈마동 293번지 14호
갈마동 293번지 2호
갈마동 295번지 20호
갈마동 295번지 4호
갈마동 297번지 26호 사랑아트빌라
갈마동 306번지 29호
갈마동 310번지 15호
갈마동 310번지 1호
갈마동 313번지 12호
갈마동 313번지 1호 102호
갈마동 313번지 8호
갈마동 313번지 9호
갈마동 316번지 6호 유원빌
갈마동 319번지 5호
갈마동 321번지 12호
갈마동 328번지 35호
갈마동 333번지 6호 1층
갈마동 336번지 22호
갈마동 337번지 15호
갈마동 338번지 1호
갈마동 339번지 1호
갈마동 339번지 3호
갈마동 343번지 30호
갈마동 344번지 1호
갈마동 352번지 1호
갈마동 352번지 4호
갈마동 362번지 2호
갈마동 366번지 10호
갈마동 367번지 21호
갈마동 368번지 14호
갈마동 378번지 18호
갈마동 378번지 23호
갈마동 388번지 31호
갈마동 391번지 22호
갈마동 393번지 27호
갈마동 402번지 2호 쌍용아파트
갈마동 405번지 10호
갈마동 427번지 79호
갈마동 430번지 9호
갈마동 309-3번지
관저1동 2042번지 3호
관저1동 83번지 6호
관

In [21]:
for i in error_index_ho:
    for j in df_error['주소'][i].split(' '):
        if '번지' in j:
            df_error['주소'][i] = df_error['주소'][i].replace('번지 ','-') # 'XXX번지 XX호'를 'XXX-XX'로 변환
            df_error['주소'][i] = df_error['주소'][i].replace('호', '')
            print(df_error['주소'][i])

가수원동 656-27
가수원동 656-62
가장동 27-18
가장동 30-12
가장동 42-15 1층
갈마1동 1427-102
갈마1동 262-12
갈마1동 262-4
갈마1동 264-5
갈마1동 285-8
갈마1동 290-12 1층
갈마1동 307-27 102
갈마1동 316-4
갈마1동 366-23 1층
갈마2동 343-27
갈마2동 351-2 한국도로공사조합아파트상가 104
갈마2동 394-18 1층
갈마2동 398-4
갈마2동 404-9
갈마동 261-14 .261-17
갈마동 261-26
갈마동 263-29
갈마동 265-34
갈마동 266-85 102
갈마동 271-1
갈마동 293-14
갈마동 293-2
갈마동 295-20
갈마동 295-4
갈마동 297-26 사랑아트빌라
갈마동 306-29
갈마동 310-15
갈마동 310-1
갈마동 313-12
갈마동 313-1 102
갈마동 313-8
갈마동 313-9
갈마동 316-6 유원빌
갈마동 319-5
갈마동 321-12
갈마동 328-35
갈마동 333-6 1층
갈마동 336-22
갈마동 337-15
갈마동 338-1
갈마동 339-1
갈마동 339-3
갈마동 343-30
갈마동 344-1
갈마동 352-1
갈마동 352-4
갈마동 362-2
갈마동 366-10
갈마동 367-21
갈마동 368-14
갈마동 378-18
갈마동 378-23
갈마동 388-31
갈마동 391-22
갈마동 393-27
갈마동 402-2 쌍용아파트
갈마동 405-10
갈마동 427-79
갈마동 430-9
갈마동 309-3번지
관저1동 2042-3
관저1동 83-6
관저동 1557-4
관저동 1558-6
관저동 1564-1
관저동 1569-14
관저동 1612-1
관저동 1943-7
관저동 1962-9
관저동 1967-1
관저동 1975-1
괴곡동 894-8
괴정동 10-6
괴정동 110-21
괴정동 110-8 부원빌라
괴정동 113-12
괴정동 125-25
괴정동 15-9
괴정동 423-19
괴정동 423-27
괴정동 4

In [22]:
for i in error_index_ho:
    for j in df_error['주소'][i].split(' '):
        if '번지' in j:
            df_error['주소'][i] = df_error['주소'][i].replace('번지','') #번지가 두 번 들어간 경우, 두번째 번지는 없앰
            print(df_error['주소'][i])

갈마동 309-3


In [23]:
for i in error_index_ho:
    df_error['주소'][i] = df_error['주소'][i].replace('호','') # '호' 제거
    print(df_error['주소'][i])

가수원동 1031 402 일부
가수원동 1032 201~203 
가수원동 1033 401402 
가수원동 1037 501502 
가수원동 1050 109 
가수원동 1051 좋은빌딩 301-B
가수원동 1052 401501502503 
가수원동 1058 106 106-1 
가수원동 1058 2층 201~204(유니온빌딩) 
가수원동 1061 108 
가수원동 1061 리더스프라자에이 106 
가수원동 1061 리더스프라자에이 115 
가수원동 1062 206 
가수원동 1320 2층 201 202 203 204 
가수원동 1320 2층 205206 
가수원동 174-16
가수원동 178-5 
가수원동 188-1
가수원동 429-1
가수원동 484 
가수원동 656-27
가수원동 656-62
가수원동 663-2 
가수원동 771-1
가수원동 771-2
가수원동 785-10
가수원동 790-1
가수원동 847 성광타워 107
가수원동 849 104(우보메디피아) 
가수원동 849 401
가수원동 967 라이프상가 126 
가수원동 375
가장동 27-18
가장동 30-15
가장동 30-12
가장동 39-22
가장동 40-40
가장동 42-15 1층
가장로 106 202-2 (가장동 삼성 래미안아파트)
갈마1동 1427-102
갈마1동 262-12
갈마1동 262-4
갈마1동 264-5
갈마1동 285-8
갈마1동 290-12 1층
갈마1동 307-27 102
갈마1동 316-4
갈마1동 366-23 1층
갈마2동 343-27
갈마2동 351-2 한국도로공사조합아파트상가 104
갈마2동 394-18 1층
갈마2동 398-4
갈마2동 404-9
갈마동 114-7
갈마동 261-10 
갈마동 261-14 .261-17
갈마동 261-26
갈마동 263-4
갈마동 263-29
갈마동 265-5
갈마동 265-34
갈마동 266-85 102
갈마동 268-20
갈마동 268-4
갈마동 271-25
갈마동 271-27
갈마동 271-1
갈마동 281-4 
갈마동 293-9 

In [24]:
for i in error_index_ho:
    df_error['주소'][i] = df_error['주소'][i].split(' ')[0]+' '+df_error['주소'][i].split(' ')[1] # 앞의 두 단어만 추출
    print(df_error['주소'][i])

가수원동 1031
가수원동 1032
가수원동 1033
가수원동 1037
가수원동 1050
가수원동 1051
가수원동 1052
가수원동 1058
가수원동 1058
가수원동 1061
가수원동 1061
가수원동 1061
가수원동 1062
가수원동 1320
가수원동 1320
가수원동 174-16
가수원동 178-5
가수원동 188-1
가수원동 429-1
가수원동 484
가수원동 656-27
가수원동 656-62
가수원동 663-2
가수원동 771-1
가수원동 771-2
가수원동 785-10
가수원동 790-1
가수원동 847
가수원동 849
가수원동 849
가수원동 967
가수원동 375
가장동 27-18
가장동 30-15
가장동 30-12
가장동 39-22
가장동 40-40
가장동 42-15
가장로 106
갈마1동 1427-102
갈마1동 262-12
갈마1동 262-4
갈마1동 264-5
갈마1동 285-8
갈마1동 290-12
갈마1동 307-27
갈마1동 316-4
갈마1동 366-23
갈마2동 343-27
갈마2동 351-2
갈마2동 394-18
갈마2동 398-4
갈마2동 404-9
갈마동 114-7
갈마동 261-10
갈마동 261-14
갈마동 261-26
갈마동 263-4
갈마동 263-29
갈마동 265-5
갈마동 265-34
갈마동 266-85
갈마동 268-20
갈마동 268-4
갈마동 271-25
갈마동 271-27
갈마동 271-1
갈마동 281-4
갈마동 293-9
갈마동 293-14
갈마동 293-2
갈마동 295-20
갈마동 295-4
갈마동 297-26
갈마동 305-340
갈마동 306-29
갈마동 310-15
갈마동 310-1
갈마동 313-12
갈마동 313-1
갈마동 313-8
갈마동 313-9
갈마동 316-4
갈마동 316-6
갈마동 319-5
갈마동 321-12
갈마동 328-35
갈마동 333-6
갈마동 336-22
갈마동 337-15
갈마동 338-1
갈마동 339-1
갈마동 339-3
갈마동 343-46
갈마동 343-

In [25]:
df_error

,광역시,구,구분,시설명,주소
6,대전광역시,서구,일반음식점 및 휴게음식점,뚱보집가수원점,가수원동 1008 3
12,대전광역시,서구,일반음식점 및 휴게음식점,바담식탁,가수원동 1019
13,대전광역시,서구,일반음식점 및 휴게음식점,왕돌구이삼겹살왕삼이,가수원동 1020 1층
14,대전광역시,서구,일반음식점 및 휴게음식점,푸른바다참치연어,가수원동 1023
15,대전광역시,서구,일반음식점 및 휴게음식점,못난이꽈배기가수원점,가수원동 1024 RiverTree
...,...,...,...,...,...
15019,대전광역시,서구,담배판매업소 및 담배자동판매기,청마트,흑석동 381
15020,대전광역시,서구,일반음식점 및 휴게음식점,서울식당,흑석동 389-2
15022,대전광역시,서구,담배판매업소 및 담배자동판매기,물안유원지,흑석동 432-3
15034,대전광역시,서구,연면적 1천제곱미터 이상 건축물,우리요양원,흑석동 903 외2필지


In [26]:
error_index1 = []

for i in error_index_ho:
    result = kakao.search_address(df_error['주소'][i])
    if len(result['documents']) == 0:
        error_index1.append(i)

print(len(error_index1))
# 유효하지 않은 주소 중 '호' 포함했던 주소 552개가 66개로 감소
# 즉, 486개의 주소 살려냄

66


In [27]:
df_error = df_error.drop(error_index1, axis=0)
print(df_error.shape)
# 유효하지 않은 66개의 주소 행 삭제

(883, 5)


### 기타 경우의 데이터 정제

In [28]:
error_index_else = df_error.index.tolist()
df_error.loc[error_index_else]

,광역시,구,구분,시설명,주소
6,대전광역시,서구,일반음식점 및 휴게음식점,뚱보집가수원점,가수원동 1008 3
12,대전광역시,서구,일반음식점 및 휴게음식점,바담식탁,가수원동 1019
13,대전광역시,서구,일반음식점 및 휴게음식점,왕돌구이삼겹살왕삼이,가수원동 1020 1층
14,대전광역시,서구,일반음식점 및 휴게음식점,푸른바다참치연어,가수원동 1023
15,대전광역시,서구,일반음식점 및 휴게음식점,못난이꽈배기가수원점,가수원동 1024 RiverTree
...,...,...,...,...,...
15004,대전광역시,서구,학원,둔산삼육외국어학원,한밭대로707번길 31 4일부5층 (월평동)
15020,대전광역시,서구,일반음식점 및 휴게음식점,서울식당,흑석동 389-2
15022,대전광역시,서구,담배판매업소 및 담배자동판매기,물안유원지,흑석동 432-3
15034,대전광역시,서구,연면적 1천제곱미터 이상 건축물,우리요양원,흑석동 903 외2필지


In [29]:
for i in error_index_else:
    if '가수원동' in df_error['주소'][i]:
        df_error['주소'][i] = df_error['주소'][i].replace('가수원동', '괴곡동') 
        #'가수원동'은 행정동명, '괴곡동'은 법정동명. 행정동명으로 검색 시 결과가 안 뜨는 문제 해결(15번째 행)

print(df_error['주소'])

6                      괴곡동 1008 3 
12                       괴곡동 1019 
13                    괴곡동 1020 1층 
14                       괴곡동 1023 
15             괴곡동 1024 RiverTree 
                   ...            
15004    한밭대로707번길 31  4일부5층 (월평동)
15020                   흑석동 389-2 
15022                    흑석동 432-3
15034                 흑석동 903 외2필지
15056                     흑석동 15블럭
Name: 주소, Length: 883, dtype: object


In [30]:
for i in error_index_else:
    if '번지' in df_error['주소'][i]:
        df_error['주소'][i] = df_error['주소'][i].replace('번지', '') #'번지' 없앰

print(df_error['주소'])

6                      괴곡동 1008 3 
12                       괴곡동 1019 
13                    괴곡동 1020 1층 
14                       괴곡동 1023 
15             괴곡동 1024 RiverTree 
                   ...            
15004    한밭대로707번길 31  4일부5층 (월평동)
15020                   흑석동 389-2 
15022                    흑석동 432-3
15034                 흑석동 903 외2필지
15056                     흑석동 15블럭
Name: 주소, Length: 883, dtype: object


In [31]:
for i in error_index_else:
    if bool(len(df_error['주소'][i].split(' '))-1)==False:
        print(i)

2091
11319
13148


In [32]:
df_error['주소'][2091] # 갈마로31인지 갈마로312인지(둘다 존재하는 주소) 구분불가하므로 drop

'갈마로3120(가장동)'

In [33]:
df_error['주소'][11319] # 존재하지 않는 주소

'복수동로100-19(복수동)'

In [34]:
df_error['주소'][13148] # 존재하지 않는 주소

'유등로18-7(탄방동)'

In [35]:
df_error = df_error.drop([2091, 11319, 13148], axis=0)

In [36]:
error_index_else = df_error.index.tolist()

In [37]:
for i in error_index_else:
    df_error['주소'][i] = df_error['주소'][i].split(' ')[0]+' '+df_error['주소'][i].split(' ')[1] # 앞의 두 단어만 추출
    print(df_error['주소'][i])

# IndexError: list index out of range -> 바로 위의 2091, 11319, 13148번째 행 제거하는 것으로 해결

괴곡동 1008
괴곡동 1019
괴곡동 1020
괴곡동 1023
괴곡동 1024
괴곡동 1025
괴곡동 1026
괴곡동 1028
괴곡동 1028
괴곡동 1028
괴곡동 1028
괴곡동 1028
괴곡동 1028
괴곡동 1028
괴곡동 1028-0
괴곡동 1029
괴곡동 1029
괴곡동 1029
괴곡동 1029
괴곡동 1029
괴곡동 1029
괴곡동 1029-0
괴곡동 1030
괴곡동 1030
괴곡동 1030
괴곡동 1030
괴곡동 1030
괴곡동 1030
괴곡동 1030-0
괴곡동 1031
괴곡동 1031
괴곡동 1031
괴곡동 1031
괴곡동 1031
괴곡동 1031-0
괴곡동 1031
괴곡동 1032
괴곡동 1032
괴곡동 1032
괴곡동 1032
괴곡동 1032
괴곡동 1032
괴곡동 1032-0
괴곡동 1032-602
괴곡동 1033
괴곡동 1037
괴곡동 1042
괴곡동 1050
괴곡동 1050
괴곡동 1050
괴곡동 1050
괴곡동 1050
괴곡동 1050
괴곡동 1050-0
괴곡동 1051
괴곡동 1051
괴곡동 1051
괴곡동 1051
괴곡동 1051-0
괴곡동 1051
괴곡동 1052
괴곡동 1052
괴곡동 1052
괴곡동 1052-0
괴곡동 1056
괴곡동 1056
괴곡동 1056
괴곡동 1056-0
괴곡동 1057
괴곡동 1057
괴곡동 1057-0
괴곡동 1058
괴곡동 1058
괴곡동 1058
괴곡동 1058
괴곡동 1058
괴곡동 1058
괴곡동 1058-0
괴곡동 1061
괴곡동 1061
괴곡동 1061
괴곡동 1061
괴곡동 1061-0
괴곡동 1061
괴곡동 1062
괴곡동 1062
괴곡동 1062
괴곡동 1062
괴곡동 1062
괴곡동 1062
괴곡동 1062
괴곡동 1062-0
괴곡동 1068
괴곡동 1069
괴곡동 1071
괴곡동 1073
괴곡동 1074
괴곡동 1075
괴곡동 1089
괴곡동 1091
괴곡동 1100
괴곡동 1116
괴곡동 1122
괴곡동 1136
괴곡동 1137
괴곡동 1143
괴곡동 1161
괴곡동 116

In [38]:
for i in error_index_else:
    if '번지' in df_error['주소'][i]:
        df_error['주소'][i] = df_error['주소'][i].replace('-0', '') #'-0' 없앰
        df_error['주소'][i] = df_error['주소'][i].replace('-602', '') #'-602' 없앰
print(df_error['주소'])

6            괴곡동 1008
12           괴곡동 1019
13           괴곡동 1020
14           괴곡동 1023
15           괴곡동 1024
             ...     
15004    한밭대로707번길 31
15020       흑석동 389-2
15022       흑석동 432-3
15034         흑석동 903
15056        흑석동 15블럭
Name: 주소, Length: 880, dtype: object


In [39]:
error_index2 = []

for i in error_index_else:
    result = kakao.search_address(df_error['주소'][i])
    if len(result['documents']) == 0:
        error_index2.append(i)

print(len(error_index2))

232


In [40]:
df_error = df_error.drop(error_index2, axis=0)
print(df_error.shape)

# 위에서 1차 정제 후 검색되지 않는 주소 데이터 955개 존재하였으나,
# 최종적으로 648개의 데이터는 살릴 수 있게 되었음
# 즉 307개의 데이터 삭제(전체 데이터 수의 약 2%)

(648, 5)


## 데이터 병합

In [41]:
df1 = pd.concat([df, df_error])
df1

,광역시,구,구분,시설명,주소
0,대전광역시,서구,일반음식점 및 휴게음식점,레몽이퐁당,가수원동 1001
1,대전광역시,서구,일반음식점 및 휴게음식점,소쿠리족발보쌈,가수원동 1002
2,대전광역시,서구,일반음식점 및 휴게음식점,남다른맥주가게,가수원동 1003
3,대전광역시,서구,일반음식점 및 휴게음식점,비어뱅크,가수원동 1004
4,대전광역시,서구,일반음식점 및 휴게음식점,구이구이삼겹살,가수원동 1005
...,...,...,...,...,...
14940,대전광역시,서구,담배판매업소 및 담배자동판매기,NaN,탄방동 1481-2
15004,대전광역시,서구,학원,둔산삼육외국어학원,한밭대로707번길 31
15022,대전광역시,서구,담배판매업소 및 담배자동판매기,물안유원지,흑석동 432-3
15034,대전광역시,서구,연면적 1천제곱미터 이상 건축물,우리요양원,흑석동 903


In [42]:
df1 = df1.reset_index(drop=True)
df1

,광역시,구,구분,시설명,주소
0,대전광역시,서구,일반음식점 및 휴게음식점,레몽이퐁당,가수원동 1001
1,대전광역시,서구,일반음식점 및 휴게음식점,소쿠리족발보쌈,가수원동 1002
2,대전광역시,서구,일반음식점 및 휴게음식점,남다른맥주가게,가수원동 1003
3,대전광역시,서구,일반음식점 및 휴게음식점,비어뱅크,가수원동 1004
4,대전광역시,서구,일반음식점 및 휴게음식점,구이구이삼겹살,가수원동 1005
...,...,...,...,...,...
14751,대전광역시,서구,담배판매업소 및 담배자동판매기,NaN,탄방동 1481-2
14752,대전광역시,서구,학원,둔산삼육외국어학원,한밭대로707번길 31
14753,대전광역시,서구,담배판매업소 및 담배자동판매기,물안유원지,흑석동 432-3
14754,대전광역시,서구,연면적 1천제곱미터 이상 건축물,우리요양원,흑석동 903


## 동, 위경도 추출

In [43]:
add_list = df1['주소']

In [44]:
for i in range(len(add_list)):
    if bool(kakao.search_address(add_list[i])['documents']) == False:
        print(i)

In [53]:
for i in range(len(add_list)):
    if bool(kakao.search_address(add_list[i])['documents']) == True:
        if bool(kakao.search_address(add_list[i])['documents'][0]['address']) == False:
            print(i)

4250
14497
14568


In [55]:
kakao.search_address(add_list[4250])['documents'][0]['road_address']['x']

'127.391721553269'

In [56]:
kakao.search_address(add_list[14497])['documents'][0]['road_address']['x']

'127.392420520302'

In [57]:
kakao.search_address(add_list[14568])['documents'][0]['road_address']['x']

'127.394060064738'

In [58]:
df1['동'] = np.nan
df1['위도'] = np.nan
df1['경도'] = np.nan
df1

,광역시,구,구분,시설명,주소,동,위도,경도
0,대전광역시,서구,일반음식점 및 휴게음식점,레몽이퐁당,가수원동 1001,NaN,NaN,NaN
1,대전광역시,서구,일반음식점 및 휴게음식점,소쿠리족발보쌈,가수원동 1002,NaN,NaN,NaN
2,대전광역시,서구,일반음식점 및 휴게음식점,남다른맥주가게,가수원동 1003,NaN,NaN,NaN
3,대전광역시,서구,일반음식점 및 휴게음식점,비어뱅크,가수원동 1004,NaN,NaN,NaN
4,대전광역시,서구,일반음식점 및 휴게음식점,구이구이삼겹살,가수원동 1005,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
14751,대전광역시,서구,담배판매업소 및 담배자동판매기,NaN,탄방동 1481-2,NaN,NaN,NaN
14752,대전광역시,서구,학원,둔산삼육외국어학원,한밭대로707번길 31,NaN,NaN,NaN
14753,대전광역시,서구,담배판매업소 및 담배자동판매기,물안유원지,흑석동 432-3,NaN,NaN,NaN
14754,대전광역시,서구,연면적 1천제곱미터 이상 건축물,우리요양원,흑석동 903,NaN,NaN,NaN


In [91]:
for i in range(len(add_list)):
    if i == 4250 | 14497 | 14568:
        df1['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['road_address']['x']
        df1['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['road_address']['y']
    else:
        if bool(kakao.search_address(add_list[i])['documents'][0]['x']) == False:
            df1['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['x']
            df1['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['y']
        else:
            df1['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
            df1['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y'] 
        
# else문에서 TypeError: 'NoneType' object is not subscriptable
# 해결 위해 

In [89]:
for i in range(len(add_list)):
    if i == 4250 | 14497 | 14568:
        df1['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['road_address']['region_3depth_name']
    elif bool(kakao.search_address(add_list[i])['documents'][0]['address']) == True:
        df1['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_name']
    else:
        df1['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['road_address']['region_3depth_name']

In [92]:
df1

,광역시,구,구분,시설명,주소,동,위도,경도
0,대전광역시,서구,일반음식점 및 휴게음식점,레몽이퐁당,가수원동 1001,괴곡동,36.2848113372001,127.350180559864
1,대전광역시,서구,일반음식점 및 휴게음식점,소쿠리족발보쌈,가수원동 1002,괴곡동,36.2845701542483,127.349819839623
2,대전광역시,서구,일반음식점 및 휴게음식점,남다른맥주가게,가수원동 1003,괴곡동,36.2844311767786,127.346730593623
3,대전광역시,서구,일반음식점 및 휴게음식점,비어뱅크,가수원동 1004,괴곡동,36.2827623489009,127.34930476633
4,대전광역시,서구,일반음식점 및 휴게음식점,구이구이삼겹살,가수원동 1005,괴곡동,36.2826258980156,127.348350922703
...,...,...,...,...,...,...,...,...
14751,대전광역시,서구,담배판매업소 및 담배자동판매기,NaN,탄방동 1481-2,탄방동,36.3452802490573,127.401200478326
14752,대전광역시,서구,학원,둔산삼육외국어학원,한밭대로707번길 31,월평동,36.3592940061448,127.377255402669
14753,대전광역시,서구,담배판매업소 및 담배자동판매기,물안유원지,흑석동 432-3,흑석동,37.134371806837,128.246234058113
14754,대전광역시,서구,연면적 1천제곱미터 이상 건축물,우리요양원,흑석동 903,흑석동,36.255665875454,127.342132984743


In [93]:
df1 = df1[['광역시','구','동','주소','구분','시설명','위도','경도']]
df1

,광역시,구,동,주소,구분,시설명,위도,경도
0,대전광역시,서구,괴곡동,가수원동 1001,일반음식점 및 휴게음식점,레몽이퐁당,36.2848113372001,127.350180559864
1,대전광역시,서구,괴곡동,가수원동 1002,일반음식점 및 휴게음식점,소쿠리족발보쌈,36.2845701542483,127.349819839623
2,대전광역시,서구,괴곡동,가수원동 1003,일반음식점 및 휴게음식점,남다른맥주가게,36.2844311767786,127.346730593623
3,대전광역시,서구,괴곡동,가수원동 1004,일반음식점 및 휴게음식점,비어뱅크,36.2827623489009,127.34930476633
4,대전광역시,서구,괴곡동,가수원동 1005,일반음식점 및 휴게음식점,구이구이삼겹살,36.2826258980156,127.348350922703
...,...,...,...,...,...,...,...,...
14751,대전광역시,서구,탄방동,탄방동 1481-2,담배판매업소 및 담배자동판매기,NaN,36.3452802490573,127.401200478326
14752,대전광역시,서구,월평동,한밭대로707번길 31,학원,둔산삼육외국어학원,36.3592940061448,127.377255402669
14753,대전광역시,서구,흑석동,흑석동 432-3,담배판매업소 및 담배자동판매기,물안유원지,37.134371806837,128.246234058113
14754,대전광역시,서구,흑석동,흑석동 903,연면적 1천제곱미터 이상 건축물,우리요양원,36.255665875454,127.342132984743


## 데이터 저장

In [94]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14756 entries, 0 to 14755
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   광역시     14756 non-null  object
 1   구       14756 non-null  object
 2   동       14756 non-null  object
 3   주소      14756 non-null  object
 4   구분      14756 non-null  object
 5   시설명     13657 non-null  object
 6   위도      14756 non-null  object
 7   경도      14756 non-null  object
dtypes: object(8)
memory usage: 922.4+ KB


In [100]:
df1.to_csv("C:/Python/NonSmoking_Project/data/금연구역지정현황_서구_위경도추출.csv", index=False, encoding='cp949')